<a href="https://colab.research.google.com/github/marcinwolter/NumericalAnalysis2020/blob/master/lab5_newton_multivariate_aitken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import numpy as np

# Example:

def jacobian_example(xy):
    x, y = xy
    return [[1, 2],
            [2*x, 8*y]]

def function_example(xy):
    x, y = xy
    return [x + 2*y - 2, x**2 + 4*y**2 - 4]




# Another  exercise:
def function_exercise(xyz):
    x, y, z = xyz
    return [x + y + z - 3,
            x**2 + y**2 + z**2 - 5,
            np.exp(x) + x*y - x*z - 1]

def jacobian_exercise(xyz):
    x, y, z = xyz
    return [[1, 1, 1],
            [2*x, 2*y, 2*z],
            [np.exp(x) + y - z, x, -x]]

# slow convergence
def jacobian_slow(xy):
    x, y = xy
    return [[2*x, 3*3*y**2],
            [1, 1]]

def function_slow(xy):
    x, y = xy
    return [x**2 + 3*y**3 , x + y ]

    
# iterative Newton's method
def iterative_newton(fun, x_init, jacobian):
    max_iter = 150
    epsilon = 1e-8

    x_last = x_init

    for k in range(max_iter):
        # Solve J(xn)*( xn+1 - xn ) = -F(xn):
        J = np.array(jacobian(x_last))
        F = np.array(fun(x_last))


        # inverted Jacobian
        Jinv = np.linalg.inv(J)
        #  x(n+1) = x(n) − J^(−1)(x(n))*F(x(n)).
        #  np.dot - array/vector multiplication
        diff = np.dot(Jinv,F)
        x_last = x_last - diff
        
        print(x_last)        

        # Stop condition:
        if np.linalg.norm(diff) < epsilon:
            print('convergence!, nre iter:', k )
            break

    else: # only if the for loop end 'naturally'
        print('not converged')

    return x_last


# iterative Newton's method with Aitken's acceleration
def iterative_aitken(fun, x_init, jacobian):
    max_iter = 50
    epsilon = 1e-8

    # Aitken's acceleration from http://www.zen89632.zen.co.uk/R/EM_Aitken/em_aitken.pdf

    loop_break = 0


    
    l = len(x_init)
    X = np.zeros([max_iter+l+2,l])
    X[0] = x_init

    for n in range(0, max_iter, l):
      if loop_break:
        break
      for k in range (0,l):
        # Solve J(xn)*( xn+1 - xn ) = -F(xn):
        J = np.array(jacobian(X[n+k]))
        F = np.array(fun(X[n+k]))


        # inverted Jacobian
        Jinv = np.linalg.inv(J)
        #  x(n+1) = x(n) − J^(−1)(x(n))*F(x(n)).
        #  np.dot - array/vector multiplication
        diff = np.dot(Jinv,F)
        X[n+k+1] = X[n+k] - diff

        
        print(X[n+k+1])   

        
      X1=np.zeros([l,l])
      X2=np.zeros([l,l])
      X3=np.zeros([l,l])

      for i in range(l):
          for j in range(l):
            X1[j,i] = X[n+k-1-i,j] 
            X2[j,i] = X[n+k-i,j] 
            X3[j,i] = X[n+k+1-i,j] 
        
      #print("X1 ",X1)
      #print("X2 ",X2)
      #print("X3 ",X3)
      #print("X2-X1 ",np.subtract(X2,X1)," det = ", np.linalg.det(np.subtract(X2,X1)))
      if abs( np.linalg.det(np.subtract(X2,X1)) ) > 1e-16:
           Lambda = np.dot(np.subtract(X3,X2),np.linalg.inv(np.subtract(X2,X1)))
           #print("Lambda = ",Lambda)
           XX1 = np.linalg.inv(np.subtract(np.identity(l),Lambda))
           XX2 = np.subtract(X[n+1],np.dot(Lambda,X[n]))
           XX  = np.dot(XX1, XX2)
           #print(" XX0 = ",np.subtract(np.identity(l),Lambda),np.identity(l),Lambda)
           #print(" XX1 = ",XX1)
           #print(" XX2 = ",XX2)
           print(" Without acceleration = ",X[n+k+1])
           print(" Aitken's accelerated = ",XX)
           X[n+k+1] = XX

          


      # Stop condition:
      if np.linalg.norm(diff) < epsilon:
            print('convergence!, nre iter:', n )
            X_out = X[n+k+1]
            loop_break = 1
            break

    else: # only if the for loop end 'naturally'
        print('not converged')
        X_out = X[n+k+1]

    return X_out





# slightly different implementation solving a set of linear equations
def iterative_newton_v2(fun, x_init, jacobian):
    max_iter = 50
    epsilon = 1e-8

    x_last = x_init

    for k in range(max_iter):
        # Solve J(xn)*( xn+1 - xn ) = -F(xn):
        J = np.array(jacobian(x_last))
        F = np.array(fun(x_last))

        diff = np.linalg.solve( J, -F )
        x_last = x_last + diff

        # Stop condition:
        if np.linalg.norm(diff) < epsilon:
            print('convergence!, nre iter:', k )
            break

    else: # only if the for loop end 'naturally'
        print('not converged')

    return x_last



# For the exercice:
x_sol = iterative_newton(function_exercise, [2.0,1.0,2.0], jacobian_exercise)
print('solution exercice:', x_sol )
print('F(sol)', function_exercise(x_sol) )
print(" ")



# second version
x_sol = iterative_newton_v2(function_exercise, [2.0,1.0,2.0], jacobian_exercise)
print('solution exercise:', x_sol )
print('F(sol)', function_exercise(x_sol) )
print(" ")

# slow convergence
x_sol = iterative_newton(function_slow, [2.0,1.0], jacobian_slow)
print('solution slow:', x_sol )
print('F(sol)', function_slow(x_sol) )
print(" ")

x_sol = iterative_aitken(function_slow, [2.0,1.0], jacobian_slow)
print('solution slow:', x_sol )
print('F(sol)', function_slow(x_sol) )
print(" ")




# For the example:
x_sol = iterative_newton(function_example, [1.0,2.0], jacobian_example)
print('solution example:', x_sol )
print( function_example(x_sol) )

[ 1.95362338 -1.          2.04637662]
[ 1.49651235 -0.32885865  1.83234629]
[ 1.28066846 -0.13217078  1.85150232]
[ 1.2279083  -0.09540156  1.86749326]
[ 1.22440996 -0.09314308  1.86873313]
[ 1.22439432 -0.09313314  1.86873882]
[ 1.22439432 -0.09313314  1.86873882]
convergence!, nre iter: 6
solution exercice: [ 1.22439432 -0.09313314  1.86873882]
F(sol) [0.0, 0.0, 0.0]
 
convergence!, nre iter: 6
solution exercise: [ 1.22439432 -0.09313314  1.86873882]
F(sol) [0.0, 0.0, -4.440892098500626e-16]
 
[-2.  2.]
[-1.3  1.3]
[-0.8350365  0.8350365]
[-0.52743867  0.52743867]
[-0.32556764  0.32556764]
[-0.19503289  0.19503289]
[-0.11271012  0.11271012]
[-0.06267652  0.06267652]
[-0.03363636  0.03363636]
[-0.01755518  0.01755518]
[-0.0089918  0.0089918]
[-0.00455418  0.00455418]
[-0.00229233  0.00229233]
[-0.00115007  0.00115007]
[-0.00057602  0.00057602]
[-0.00028826  0.00028826]
[-0.00014419  0.00014419]
[-7.21113751e-05  7.21113751e-05]
[-3.60595863e-05  3.60595863e-05]
[-1.80307682e-05  1.803